In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import matplotlib.image as mpimg
from PIL import Image

from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold

from sklearn.mixture import GaussianMixture
from sklearn.svm import SVC

## Task 1. Gaussian Mixture Model

In [2]:
FER2013_AU = pd.read_csv('Fer2013/fer2013.csv')
FER2013_AU

,file_name,confidence,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,label
0,Training_983506.jpg,0.875,0.00,0.00,3.42,0.02,0.21,1.33,0.17,1.07,...,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,angry
1,Training_2944904.jpg,0.925,0.00,0.00,0.00,0.00,0.14,1.35,0.66,0.76,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,angry
2,Training_3629189.jpg,0.925,1.13,0.00,0.00,0.05,0.00,0.00,0.00,0.67,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,angry
3,Training_1477608.jpg,0.925,0.00,0.00,0.00,0.66,0.00,0.00,0.00,0.00,...,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,angry
4,Training_1193977.jpg,0.875,0.01,0.00,0.00,0.47,0.11,2.21,0.77,0.07,...,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,angry
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
807,Training_1820471.jpg,0.925,1.18,0.81,1.29,1.18,0.00,0.96,0.00,0.00,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,surprise
808,Training_2594780.jpg,0.975,0.00,0.00,0.00,0.00,0.65,1.94,0.03,1.52,...,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,surprise
809,Training_3297674.jpg,0.925,1.55,1.22,0.90,2.52,0.00,0.00,0.00,0.00,...,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,surprise
810,Training_734790.jpg,0.975,0.99,1.10,0.84,0.48,0.00,0.11,0.00,0.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,surprise


In [3]:
FER2013_AU_X = FER2013_AU.drop(['file_name','confidence','label'],axis=1)
FER2013_AU_y = FER2013_AU['label']

In [4]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
accuracy = []
precision = []
recall = []
f1 = []
matrix = []

for train_index, test_index in kf.split(FER2013_AU_X):
    train_X, train_y = FER2013_AU_X.iloc[train_index,:], FER2013_AU_y[train_index]
    test_X, test_y = FER2013_AU_X.iloc[test_index,:], FER2013_AU_y[test_index]
    gmm = GaussianMixture(n_components=7, random_state=0)
    gmm.fit(train_X, train_y)
    test_y_pred = gmm.predict(test_X)
    
    map_dict = {}
    test_y_copy = test_y
    test_y_pred_copy = test_y_pred
    # Because GMM is an unsupervised model, the output information is not a label, 
    # it is necessary to obtain the corresponding information of the label and the output class
    for label in test_y.unique().tolist():
        now_label = test_y_copy[test_y_copy==label]
        now_pred = test_y_pred_copy[test_y_copy==label]
        max_num = np.argmax(np.bincount(now_pred))
        map_dict[label] = max_num
        test_y_pred_copy = test_y_pred_copy[test_y_copy!=label]
        test_y_copy = test_y_copy[test_y_copy!=label]
    
    test_y = test_y.map(map_dict)
    accuracy.append(accuracy_score(test_y, test_y_pred))
    precision.append(precision_score(test_y, test_y_pred, average="macro", zero_division=0))
    recall.append(recall_score(test_y, test_y_pred, average="macro", zero_division=0))
    f1.append(f1_score(test_y, test_y_pred, average="macro"))
    matrix.append(confusion_matrix(test_y, test_y_pred))

print(f'Accuracy in FER2013 AUs using an SVM is {np.mean(accuracy)}')
print(f'Precision in FER2013 AUs using an SVM is {np.mean(precision)}')
print(f'Recall in FER2013 AUs using an SVM is {np.mean(recall)}')
print(f'F1 in FER2013 AUs using an SVM is {np.mean(f1)}')

Accuracy in FER2013 AUs using an SVM is 0.46057714155873664
Precision in FER2013 AUs using an SVM is 0.29509365530115617
Recall in FER2013 AUs using an SVM is 0.22309616165372
F1 in FER2013 AUs using an SVM is 0.24695479272134557


In [5]:
print(f'Confusion matrix in FER2013 AUs using an SVM is')
np.round(np.mean(matrix,axis=0))

Confusion matrix in FER2013 AUs using an SVM is


array([[15.,  6.,  2.,  4.,  2.,  2.,  1.],
       [ 4., 26.,  2.,  7.,  3.,  9.,  5.],
       [ 0.,  0.,  7.,  1.,  2.,  1.,  1.],
       [ 4.,  2.,  1., 17.,  4.,  3.,  4.],
       [ 0.,  1.,  2.,  0.,  2.,  0.,  0.],
       [ 0.,  0.,  0.,  0.,  0.,  0.,  0.],
       [ 0.,  3.,  1.,  3.,  2.,  4.,  8.]])

## Task 2. Support Vector Machine

### Use raw images

In [6]:
FER2013_images_X = []
FER2013_images_y = []
for label in os.listdir('Fer2013/images'):
    for filename in os.listdir(r'./'+'Fer2013/images/'+label):
        I = Image.open('Fer2013/images/'+label+'/'+filename).convert('L')
        I = np.array(I)
        I = I.flatten()
        FER2013_images_X.append(I)
        FER2013_images_y.append(label)
FER2013_images_X = np.array(FER2013_images_X)
FER2013_images_y = np.array(FER2013_images_y)

In [7]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
accuracy = []
precision = []
recall = []
f1 = []
matrix = []

for train_index, test_index in kf.split(FER2013_images_X):
    train_X, train_y = FER2013_images_X[train_index], FER2013_images_y[train_index]
    test_X, test_y = FER2013_images_X[test_index], FER2013_images_y[test_index]
    svm = SVC(kernel='rbf')
    svm.fit(train_X, train_y)
    test_y_pred = svm.predict(test_X)
    accuracy.append(accuracy_score(test_y, test_y_pred))
    precision.append(precision_score(test_y, test_y_pred, average="macro", zero_division=0))
    recall.append(recall_score(test_y, test_y_pred, average="macro", zero_division=0))
    f1.append(f1_score(test_y, test_y_pred, average="macro"))
    matrix.append(confusion_matrix(test_y, test_y_pred))

print(f'Accuracy in FER2013 images using an SVM is {np.mean(accuracy)}')
print(f'Precision in FER2013 images using an SVM is {np.mean(precision)}')
print(f'Recall in FER2013 images using an SVM is {np.mean(recall)}')
print(f'F1 in FER2013 images using an SVM is {np.mean(f1)}')

Accuracy in FER2013 images using an SVM is 0.2761904761904762
Precision in FER2013 images using an SVM is 0.2760644035338882
Recall in FER2013 images using an SVM is 0.2867603638240948
F1 in FER2013 images using an SVM is 0.26666996236713214


In [8]:
print(f'Confusion matrix in FER2013 images using an SVM is')
np.round(np.mean(matrix,axis=0))

Confusion matrix in FER2013 images using an SVM is


array([[10.,  5.,  3.,  4.,  3.,  4.,  2.],
       [ 4., 12.,  3.,  3.,  3.,  2.,  3.],
       [ 4.,  7.,  3.,  3.,  4.,  3.,  6.],
       [ 6.,  4.,  3.,  6.,  4.,  3.,  3.],
       [ 6.,  4.,  3.,  3.,  9.,  3.,  3.],
       [ 8.,  5.,  3.,  3.,  4.,  5.,  3.],
       [ 4.,  4.,  2.,  2.,  3.,  2., 13.]])

### Use AUs

In [9]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
accuracy = []
precision = []
recall = []
f1 = []
matrix = []

for train_index, test_index in kf.split(FER2013_AU_X):
    train_X, train_y = FER2013_AU_X.iloc[train_index,:], FER2013_AU_y[train_index]
    test_X, test_y = FER2013_AU_X.iloc[test_index,:], FER2013_AU_y[test_index]
    svm = SVC(kernel='rbf')
    svm.fit(train_X, train_y)
    test_y_pred = svm.predict(test_X)
    accuracy.append(accuracy_score(test_y, test_y_pred))
    precision.append(precision_score(test_y, test_y_pred, average="macro", zero_division=0))
    recall.append(recall_score(test_y, test_y_pred, average="macro", zero_division=0))
    f1.append(f1_score(test_y, test_y_pred, average="macro"))
    matrix.append(confusion_matrix(test_y, test_y_pred))

print(f'Accuracy in FER2013 AUs using an SVM is {np.mean(accuracy)}')
print(f'Precision in FER2013 AUs using an SVM is {np.mean(precision)}')
print(f'Recall in FER2013 AUs using an SVM is {np.mean(recall)}')
print(f'F1 in FER2013 AUs using an SVM is {np.mean(f1)}')

Accuracy in FER2013 AUs using an SVM is 0.44328561690524876
Precision in FER2013 AUs using an SVM is 0.43649944121299133
Recall in FER2013 AUs using an SVM is 0.44120827012091113
F1 in FER2013 AUs using an SVM is 0.41841681599555736


In [10]:
print(f'Confusion matrix in FER2013 AUs using an SVM is')
np.round(np.mean(matrix,axis=0))

Confusion matrix in FER2013 AUs using an SVM is


array([[ 8.,  4.,  2.,  1.,  6.,  3.,  1.],
       [ 4., 10.,  0.,  1.,  2.,  2.,  1.],
       [ 4.,  2.,  1.,  1.,  4.,  4.,  5.],
       [ 2.,  0.,  0., 17.,  3.,  0.,  1.],
       [ 3.,  0.,  0.,  1., 16.,  5.,  1.],
       [ 4.,  3.,  0.,  2.,  8.,  5.,  2.],
       [ 1.,  0.,  1.,  2.,  3.,  1., 14.]])

## Task 3. Compare

In the previous task, I mainly tested two models, GMM and SVM. Use AUs, a feature extracted from images after running through OpenFace in GMM, and AUs and raw image information in SVM. 

It was found that the best results were obtained using the SVM model for the AUs data, with accuracy, precision, recall and f1 all reaching above 0.4. 

While using GMM for AUs data, accuracy can reach 0.4, but precision, recall and f1 are only about 0.25. And using SVM model for raw image information, accuracy, precision, recall and f1 are all around 0.25, which is less effective.

There are two main reasons for this in my analysis.

1. Dateset. The AUs are the features extracted by OpenFace, which are easier to be classified than the raw image information because they have been extracted.

2. Model. GMM is an unsupervised model, while SVM is a supervised model, and a supervised model can make better use of the model's labels for better classification.

## Task 4. Test

I choose SVM model.

Use AUs dataset firstly.

In [11]:
Phoebe_AU = pd.read_csv('Phoebe/phoebe.csv')
Phoebe_AU.insert(35,'AU28_c',value=0)
Phoebe_AU

,file_name,confidence,AU01_r,AU02_r,AU04_r,AU05_r,AU06_r,AU07_r,AU09_r,AU10_r,...,AU14_c,AU15_c,AU17_c,AU20_c,AU23_c,AU25_c,AU26_c,AU28_c,AU45_c,label
0,1_01.jpg,0.98,0.76,0.96,0.22,0.00,0.00,0.35,0.00,0.16,...,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0,1.0,unknown
1,2_41.jpg,0.98,0.03,0.00,1.59,0.00,0.18,0.70,0.10,0.80,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,angry
2,2_01.jpg,0.98,0.40,0.61,0.63,0.05,0.00,0.00,0.00,0.85,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,surprise
3,3_43.jpg,0.88,0.20,0.15,0.01,0.51,1.71,0.87,0.00,1.54,...,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0,0.0,happy
4,4_01.jpg,0.98,0.00,0.00,0.61,0.00,0.00,0.00,0.00,0.43,...,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0,0.0,sad
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,52_31.jpg,0.98,1.00,0.24,0.00,0.23,0.00,0.00,0.39,0.06,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,unknown
94,53_15.jpg,0.98,0.12,0.59,0.00,0.00,1.74,1.44,0.41,0.94,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,happy
95,53_02.jpg,0.77,0.16,0.22,0.45,0.31,1.60,0.77,0.00,0.96,...,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0,0.0,happy
96,17_45.jpg,0.98,0.00,0.17,0.00,0.00,0.00,0.12,0.00,0.00,...,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0,1.0,surprise


In [12]:
Phoebe_AU_X = Phoebe_AU.drop(labels=['file_name',' confidence','label'],axis=1)
Phoebe_AU_y = Phoebe_AU['label']

In [13]:
X_test = Phoebe_AU_X[Phoebe_AU_y!='unknown']
y_test = Phoebe_AU_y[Phoebe_AU_y!='unknown']

In [14]:
svm_AU = SVC(kernel='rbf')
svm_AU.fit(FER2013_AU_X, FER2013_AU_y)
Phoebe_AU_y_pred = svm_AU.predict(X_test)

In [15]:
print(f'Accuracy in Phoebe AUs using an SVM is {accuracy_score(y_test, Phoebe_AU_y_pred)}')
print(f'Precision in Phoebe AUs using an SVM is {precision_score(y_test, Phoebe_AU_y_pred, average="macro", zero_division=0)}')
print(f'Recall in Phoebe AUs using an SVM is {recall_score(y_test, Phoebe_AU_y_pred, average="macro", zero_division=0)}')
print(f'F1 score in Phoebe AUs using an SVM is {f1_score(y_test, Phoebe_AU_y_pred, average="macro")}')

Accuracy in Phoebe AUs using an SVM is 0.4
Precision in Phoebe AUs using an SVM is 0.2705936920222634
Recall in Phoebe AUs using an SVM is 0.23042687363570216
F1 score in Phoebe AUs using an SVM is 0.24535052272708865


As a comparison, use the raw image information

In [16]:
Phoebe_images_X = []
Phoebe_images_y = []
for label in os.listdir('Phoebe/images'):
    for filename in os.listdir(r'./'+'Phoebe/images/'+label):
        I = Image.open('Phoebe/images/'+label+'/'+filename).convert('L')
        I = np.array(I.resize((100,100))) # The images in the two datasets are not the same size and need to be resize
        I = I.flatten()
        Phoebe_images_X.append(I)
        Phoebe_images_y.append(label)
Phoebe_images_X = np.array(Phoebe_images_X)
Phoebe_images_y = np.array(Phoebe_images_y)

In [17]:
X_test = Phoebe_images_X[Phoebe_images_y!='unknown']
y_test = Phoebe_images_y[Phoebe_images_y!='unknown']

In [18]:
svm_images = SVC(kernel='rbf')
svm_images.fit(FER2013_images_X, FER2013_images_y)
Phoebe_images_y_pred = svm_images.predict(X_test)

In [19]:
print(f'Accuracy in Phoebe images using an SVM is {accuracy_score(y_test, Phoebe_images_y_pred)}')
print(f'Precision in Phoebe images using an SVM is {precision_score(y_test, Phoebe_images_y_pred, average="macro", zero_division=0)}')
print(f'Recall in Phoebe images using an SVM is {recall_score(y_test, Phoebe_images_y_pred, average="macro", zero_division=0)}')
print(f'F1 score in Phoebe images using an SVM is {f1_score(y_test, Phoebe_images_y_pred, average="macro")}')

Accuracy in Phoebe images using an SVM is 0.10752688172043011
Precision in Phoebe images using an SVM is 0.11281614094913328
Recall in Phoebe images using an SVM is 0.09423179160021265
F1 score in Phoebe images using an SVM is 0.07539682539682539


The SVM model was chosen to classify the Phoebe dataset using AUs and raw images, and the following conclusions were obtained.

1. In the model using the AUs dataset, the accuracy of the model did not decrease and remained around 0.4, which performed well. Unfortunately, the precision, recall and f1 have been reduced, but still higher than the model with raw images.

2. In the model with the raw images dataset, all four evaluation metrics of the model are severely degraded, and some of them are even below 0.1.

I conclude that because AUs are features extracted by a very good model, they have better robustness and generalization performance. Because the raw images are the direct features of the images without feature extraction, and the size of the images in the two datasets FER2013 and Phoebe are different, and the information is lost when scaling. This leads to better performance on the test set using the AUs model, and the raw images model has a significant decrease in metrics on the test set.

## Task 5. Classify the unknown

The predictions were made using the AUs and raw images datasets respectively, and the results were obtained as follows.

In [20]:
X_unknown = Phoebe_AU_X[Phoebe_AU_y=='unknown']
Phoebe_AU_unknown_pred = svm_AU.predict(X_unknown)
Phoebe_AU_unknown_pred

array(['sad', 'happy', 'happy', 'sad', 'happy', 'sad', 'angry', 'neutral'],
      dtype=object)

In [21]:
X_unknown = Phoebe_images_X[Phoebe_images_y=='unknown']
Phoebe_images_unknown_pred = svm_images.predict(X_unknown)
Phoebe_images_unknown_pred

array(['angry', 'neutral', 'sad', 'neutral', 'happy', 'angry', 'angry',
       'angry'], dtype='<U8')

 After my comparison, I think the predictions using the AUs dataset are more reliable.
 
Among them, 2, 3 and 5 are identified as happy, which is very accurate. 8 is identified as neutral, which is also relatively accurate. 1 is identified as sad, which is somewhat inaccurate. 4 and 6 are identified as sad, which is somewhat outrageous. 7 is identified as hangry, which is also somewhat incredible. Overall, the model is able to recognize more than half of the images more accurately, which is still relatively good.